For this to work we need to have Phantom installed (https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python) in order to be able to render javascript content

In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib
import re
import pandas as pd
from collections import defaultdict
import numpy as np

# Try 2 - 
# https://stackoverflow.com/questions/29404856/how-can-i-render-javascript-html-to-html-in-python 
# https://stackoverflow.com/questions/29858752/error-message-chromedriver-executable-needs-to-be-available-in-the-path

from selenium import webdriver
from urllib.request import urlopen
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
def read_adress_data(pickle_path='data/adresses/copenhagen_adresses.pickle', raw_path='http://dawa.aws.dk/adresser?format=csv&kommunekode=0101'):
    try: 
        print('Loading from pickle')
        df_adress = pd.read_pickle(pickle_path)
    except:
        print(f'Loading from pickle failed. Loading from source ({raw_path}) instead')
        address_path = raw_path     #address_path = 'http://dawa.aws.dk/adresser?format=json&kommunekode=0101'
        with urllib.request.urlopen(address_path) as f:
            data_binary = f.readlines() #.decode()
        data = [re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '', x.decode().strip()).split(',') for x in data_binary] # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
        df_adress = pd.DataFrame(data[1:], columns=data[0])
        df.to_pickle(pickle_path)
    return df_adress

In [3]:
df_adress = read_adress_data()

Loading from pickle


In [4]:
def get_bolig_list(municipality_code='101', pages=3):
    
    bolig_liste = defaultdict(str)

    for page in range(1, pages+1):
        url_in_scope = f'https://www.boliga.dk/resultat?sort=zipCode-a&municipality={municipality_code}&page={page}'
        print('Scraping', url_in_scope)
        try:
            page = urllib.request.urlopen(url_in_scope)
            #soup = BeautifulSoup(page, 'html.parser')
            #print(soup)

            for link in BeautifulSoup(page, 'html.parser', parse_only=SoupStrainer('a')):

                if link.has_attr('href'):
                    #print(link['href'])
                    if re.match('/bolig/\d*/.*', link['href']):
                        bolig_liste[link['href']] = defaultdict(str)


        except Exception as e:
                print("An error occured.", e)
    
    return bolig_liste

bolig_liste = get_bolig_list(municipality_code='101', pages=3)

Scraping https://www.boliga.dk/resultat?sort=zipCode-a&municipality=101&page=1
Scraping https://www.boliga.dk/resultat?sort=zipCode-a&municipality=101&page=2
Scraping https://www.boliga.dk/resultat?sort=zipCode-a&municipality=101&page=3


In [5]:
#url = bolig_url# 'http://www.google.com'
def get_html_with_js(bolig_url):
    """
    Get html with javascript data compiled
    """
    file_name = 'C:/working/ngs-lib/Boligsalg/data/url_data/data.txt'
    # Store HTML locally 
    conn = urlopen(bolig_url)
    data = conn.read()
    conn.close()
    file = open(file_name,'wb')
    file.write(data)
    file.close()

    # Compile javascript to get complete complete html
    browser = webdriver.Chrome(ChromeDriverManager().install()) #browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    browser.get('file:///'+file_name) #browser.get(url)
    html = browser.page_source
    browser.quit()
    
    return html 

bolig_urls = [f'https://www.boliga.dk{bolig}' for bolig in bolig_liste]
#bolig_url = bolig_urls[1] # 
bolig_url = 'https://www.boliga.dk/bolig/1644897/ingrid_marievej_66__12_4_2500_valby'
html = get_html_with_js(bolig_url)
soup = BeautifulSoup(html, "html.parser")


Looking for [chromedriver 80.0.3987.106 win32] driver in cache 
File found in cache by path [C:\Users\theone\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe]


In [61]:
# look_for_list = ['Marievej',
#                      '4.672',
#                      '4.945.000',
#                      #'250.000',
#                      #'43.377',
#                      'mar. 2020',
#                      'Boligen blev sat til salg',
#                      'A20',
#                      '114',
#                      '2500 Valby',
#                      '6 dage på markedet',
#                      'Se bolig hos mægler',
#                      'danbolig',
#                      '4 værelser']

# type(html)
# soup = BeautifulSoup(html, "html.parser")
# tags = {tag.name: 1 for tag in soup.find_all()}

# for t in list(tags)[3:]:
#     print('tag', t)
#     tag_contents = [str(tag_v) for tag_v in soup.find(t)]
#     for j, tag_v in enumerate(tag_contents):
#         print(j, len(tag_v))
        
#         for lookout in look_for_list:
#             if lookout in tag_v:
#                 print(' '*5, 'Interesting, found', lookout, 'in', t)

tag pre
0 285832
      Interesting, found Marievej in pre
      Interesting, found 4.672 in pre
      Interesting, found 4.945.000 in pre
      Interesting, found mar. 2020 in pre
      Interesting, found Boligen blev sat til salg in pre
      Interesting, found A20 in pre
      Interesting, found 114 in pre
      Interesting, found 2500 Valby in pre
      Interesting, found 6 dage på markedet in pre
      Interesting, found Se bolig hos mægler in pre
      Interesting, found danbolig in pre
      Interesting, found 4 værelser in pre


Manual code to determine where the relevant stuff is!

In [157]:
i = 10
print('in scope', look_for_list[i])

list_in_scope = look_for_list[i:i+1]
#list_in_scope = ['<p _ngcontent-sc39="" class="app-tooltip ng-star-inserted"><!----><!---->']

#lookout = look_for_list[0]
contents = [str(x) for x in soup.find('pre')]
for content in contents: 
    for lookout in list_in_scope:
        start = 0
        end = np.inf
        counter = 0
        while True:
            counter += 1
            start = content.find(lookout, start)
            #place = content.find(lookout, start)
            print(lookout, start) 
            print(content[start-100:start+100])
            print('')
            if start == -1:
                break
            if counter > 2:
                break
#     if lookout in content:
#         print('fun')

in scope danbolig
danbolig 124292
t-bolder px-1 ng-star-inserted" data-gtm="see_property_at_agency_btn" target="_blank" href="https://danbolig.dk/bolig/koebenhavn/2500/ejerlejlighed/069bf9-0106-069"> Se bolig hos mægler - 2 klik </a><

danbolig 124292
t-bolder px-1 ng-star-inserted" data-gtm="see_property_at_agency_btn" target="_blank" href="https://danbolig.dk/bolig/koebenhavn/2500/ejerlejlighed/069bf9-0106-069"> Se bolig hos mægler - 2 klik </a><

danbolig 124292
t-bolder px-1 ng-star-inserted" data-gtm="see_property_at_agency_btn" target="_blank" href="https://danbolig.dk/bolig/koebenhavn/2500/ejerlejlighed/069bf9-0106-069"> Se bolig hos mægler - 2 klik </a><



In [154]:
# Patterns to look for


lookout_dict = {}
lookout_dict['Adresse'] = {'start_value': ['<meta name="keywords" content="">', '<title>'],
                          'end_value': ['</title>', ' - '],
                          'name': 'Adresse',
                          'list': False,
                          'limit': 1}

lookout_dict['Basisoplysninger'] = {'start_key': ['<span _ngcontent-sc41="" class="mb-1 d-md-none font-weight-bold w-100">'],
                              'end_key': ['</span>'],
                              'start_value': ['<span _ngcontent-sc41="" class="d-md-none my-auto">'],
                              'end_value': ['</span>'],
                              'name': None, # missing, so will need to find it in data!
                              'list': False,
                              'limit': 9}


lookout_dict['Pris'] = {'start_value': ['<span class="h-md-2 h4 font-weight-bold m-0 text-nowrap">'],
                              'end_value': ['</span>'],
                              'name': 'Pris', # missing, so will need to find it in data!
                              'list': False,
                              'limit': 1}

lookout_dict['Timeline'] = {'start_value': ['<span _ngcontent-sc23="" class="d-sm-none d-table-cell timeline">'],
                              'end_value': ['</span>'],
                              'name': 'Timeline_time', # missing, so will need to find it in data!
                              'list': True,
                              'limit': 10}

lookout_dict['Timeline_action'] = {'start_value': ['<div _ngcontent-sc23="" class="mr-0 mr-sm-2"><!---->'],
                              'end_value': ['<!---->'],
                              'name': 'Timeline_action', # missing, so will need to find it in data!
                              'list': True,
                              'limit': 10}

lookout_dict['Timeline_price'] = {'start_value': ['<span _ngcontent-sc23="">'],
                              'end_value': ['<'],
                              'name': 'Timeline_price', # missing, so will need to find it in data!
                              'list': True,
                              'limit': 10}


lookout_dict['Boligtype'] = { 'start_value': ['<app-tooltip _ngcontent-sc38="" class="md-right flex-shrink-0" _nghost-sc39=""><!----><p _ngcontent-sc39="" class="app-tooltip ng-star-inserted"><!----><!---->'],
                              'end_value': ['<!---->'],
                              'name': 'Boligtype', # missing, so will need to find it in data!
                              'list': False,
                              'limit': 1}

lookout_dict['Oprettet'] = { 'start_value': ['<app-tooltip class="tooltip-nowrap" _nghost-sc39=""><!----><p _ngcontent-sc39="" class="app-tooltip ng-star-inserted"><!----><!---->', '<p _ngcontent-sc39="" class="app-tooltip ng-star-inserted"><!----><!---->'],
                              'end_value': ['<!---->'],
                              'name': 'Oprettet', # missing, so will need to find it in data!
                              'list': False,
                              'limit': 1}

lookout_dict['Markedstid'] = {'start_value': ['<use xlink:href="#icon-calendar"></use></svg><span class="text-primary h5 h-md-4 m-0">'],
                              'end_value': ['</span>'],
                              'name': 'Markedstid', # missing, so will need to find it in data!
                              'list': False,
                              'limit': 1}

lookout_dict['Link'] = {'start_value': ['<a class="btn btn-primary col-12 w-100 font-weight-bolder px-1 ng-star-inserted" data-gtm="see_property_at_agency_btn" target="_blank" href="'],
                              'end_value': ['"'],
                              'name': 'Link', # missing, so will need to find it in data!
                              'list': False,
                              'limit': 1}

# 

#lookout_dict

In [158]:
def get_start_end(lookout_starts, lookout_ends, content, start=0, end=10e6):
    """
    Get start and end of the lookout patterns
    """
    for lookout_start in lookout_starts:
        start = content.find(lookout_start, start+1)
    for e, lookout_end in enumerate(lookout_ends):
        if e == 0:
            end = content.find(lookout_end, start+len(lookout_starts[-1]))
        else: 
            end = content.rfind(lookout_end, start+len(lookout_starts[-1]), end)

    return start, end 

In [156]:
bolig_dict = {}
bolig_dict_temp = {}

#lookout = look_for_list[0]
contents = [str(x) for x in soup.find('pre')]
for content in contents: 
    for k in lookout_dict:
        print('Lookout patterns', k,  lookout_dict[k])
        start = 0
        end = len(content)
        counter = 0
        while True:
            
            if 'name' in lookout_dict[k]: 
                key_in_scope = lookout_dict[k]['name']
            
            if ('start_key' and 'end_key') in lookout_dict[k]: 
                #print('looking for key')
                lookout_starts = lookout_dict[k]['start_key'] #['<meta name="keywords" content="">', '<title>']
                lookout_ends = lookout_dict[k]['end_key']
                start, end = get_start_end(lookout_starts, lookout_ends, content, start=start, end=end)
                key_in_scope = content[start+len(lookout_starts[-1]):end].strip() # finding key from context

            if ('start_value' and 'end_value') in lookout_dict[k]: 
                #print('looking for value')
                lookout_starts = lookout_dict[k]['start_value'] #['<meta name="keywords" content="">', '<title>']
                lookout_ends = lookout_dict[k]['end_value']
                start, end = get_start_end(lookout_starts, lookout_ends, content, start=start, end=end)

            counter += 1
            if start != -1:
                print('start', start, end)
                if lookout_dict[k]['list']:
                    if key_in_scope in bolig_dict_temp:
                        bolig_dict_temp[key_in_scope].append(content[start+len(lookout_starts[-1]):end].strip())
                    else:
                        bolig_dict_temp[key_in_scope] = [content[start+len(lookout_starts[-1]):end].strip()]
                else:
                    id_counter = 2
                    if key_in_scope in bolig_dict_temp:
                        bolig_dict_temp[key_in_scope + str(id_counter)] = content[start+len(lookout_starts[-1]):end].strip()
                    else:
                        bolig_dict_temp[key_in_scope] = content[start+len(lookout_starts[-1]):end].strip()
                    id_counter += 1
            else:
                break
            if counter >= lookout_dict[k]['limit']:
                break

        #break

for k in bolig_dict_temp: 
    if len(k) < 100:
        bolig_dict[k] = bolig_dict_temp[k]
bolig_dict_temp

Lookout patterns Adresse {'start_value': ['<meta name="keywords" content="">', '<title>'], 'end_value': ['</title>', ' - '], 'name': 'Adresse', 'list': False, 'limit': 1}
start 735 780
Lookout patterns Basisoplysninger {'start_key': ['<span _ngcontent-sc41="" class="mb-1 d-md-none font-weight-bold w-100">'], 'end_key': ['</span>'], 'start_value': ['<span _ngcontent-sc41="" class="d-md-none my-auto">'], 'end_value': ['</span>'], 'name': None, 'list': False, 'limit': 9}
start 117551 117610
start 118250 118307
start 118936 118990
start 119606 119666
start 120293 120350
start 120976 121032
start 121815 121889
start 122546 122603
start 123434 123493
Lookout patterns Pris {'start_value': ['<span class="h-md-2 h4 font-weight-bold m-0 text-nowrap">'], 'end_value': ['</span>'], 'name': 'Pris', 'list': False, 'limit': 1}
start 115396 115473
Lookout patterns Timeline {'start_value': ['<span _ngcontent-sc23="" class="d-sm-none d-table-cell timeline">'], 'end_value': ['</span>'], 'name': 'Timeline_

{'Adresse': 'Ingrid Marievej 66  12. 4., 2500 Valby',
 'Boligstørrelse': '114 m²',
 'Grundstørrelse': '0 m²',
 'Værelser': '4',
 'Etage': '12. sal',
 'Byggeår': '2020',
 'Energimærke': 'A20',
 'Ejerudgift': '4.672&nbsp; kr. / md.',
 'Kælderstørrelse': '0 m²',
 'Boligareal, tinglyst': '114 m²',
 'Pris': '4.945.000&nbsp;kr.',
 'Timeline_time': ['mar. 2020'],
 'Timeline_action': ['Boligen blev sat til salg'],
 'Timeline_price': ['4.945.000&nbsp;kr.'],
 'Boligtype': 'Ejerlejlighed',
 'Oprettet': 'Oprettet 14. mar. 2020',
 'Markedstid': '6 dage på markedet',
 'Link': 'https://danbolig.dk/bolig/koebenhavn/2500/ejerlejlighed/069bf9-0106-069'}

In [104]:
# tag = soup.find('pre')
# sub_soup = BeautifulSoup(str(tag), "html.parser")
# sub_tags = {tag.name: 1 for tag in sub_soup.find_all()}
# sub_tags
# #type(html), type(soup), type(str(tag)), sub_soup

In [105]:
#str(sub_soup)

In [106]:
#subsub_soup = BeautifulSoup(sub_soup, "html.parser")

In [6]:
# for i, bolig in enumerate(bolig_liste.keys()):
#     if i == 0:
#         bolig_url = f'https://www.boliga.dk{bolig}'
#         print(bolig_url)
        
#         try:
#             page = urllib.request.urlopen(url_in_scope)
#             soup = BeautifulSoup(page, 'html.parser')
#             #print(soup)
#             #body = soup.find('body')
#             #body_no_tags = body.findChildren(recursive=False)
#             #print(body_no_tags)
#             tags = {tag.name: 1 for tag in soup.find_all()}
# #            [str(tag) for tag in soup.find_all()]
#         except Exception as e:
#             print("An error occured.", e)

https://www.boliga.dk/bolig/1616756/tyboroen_alle_91__st_2720_vanloese


In [7]:
# [str(tag_v) for tag_v in soup.find('html')][2]

# look_for_list = ['Marievej', '4.940', '4.945.000', '33.291', 'mar. 2020', 'Boligen blev sat til salg', 
#                  '120', '2500 Valby', '2500 Valby','7 dage på markedet', 'Se bolig hos mægler', 'danbolig']

# for t in list(tags):
#     #print('tag', t)
#     tag_values = [str(tag_v) for tag_v in soup.find(t)]
#     for j, tag_v in enumerate(tag_values):
#         #print(j)
#         #if j < 2:
#         for lookout in look_for_list:
#             if lookout in tag_v:
#                 print(' '*5, 'Interesting, found', lookout, 'in', t)
# #         if tag_v in
# #             print(' '*5, j, tag_v)
    

      Interesting, found Marievej in html
      Interesting, found 33.291 in html
      Interesting, found mar. 2020 in html
      Interesting, found Boligen blev sat til salg in html
      Interesting, found 120 in html
      Interesting, found 2500 Valby in html
      Interesting, found 2500 Valby in html
      Interesting, found Se bolig hos mægler in html
      Interesting, found Marievej in body
      Interesting, found 33.291 in body
      Interesting, found mar. 2020 in body
      Interesting, found 120 in body
      Interesting, found 2500 Valby in body
      Interesting, found 2500 Valby in body
      Interesting, found Marievej in body
      Interesting, found Boligen blev sat til salg in body
      Interesting, found 120 in body
      Interesting, found Se bolig hos mægler in body
      Interesting, found Marievej in app-root
      Interesting, found 33.291 in app-root
      Interesting, found mar. 2020 in app-root
      Interesting, found 120 in app-root
      Interesting, 

Some information seems to be missing. Maybe it's because some javascript is not yet calculated.


Looking for [chromedriver 80.0.3987.106 win32] driver in cache 
File found in cache by path [C:\Users\theone\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe]


In [52]:


for interest in look_for_list:
    #print(interest)
    if interest in html:
        print(interest, 'is apparant')
    else:
        print(interest, 'not found :()')

#html

Marievej is apparant
4.672 is apparant
4.945.000 is apparant
250.000 is apparant
43.377 is apparant
mar. 2020 is apparant
Boligen blev sat til salg is apparant
A20 is apparant
114 is apparant
2500 Valby is apparant
6 dage på markedet is apparant
Se bolig hos mægler is apparant
danbolig is apparant


In [ ]:
# TRY 1: Fialing.. ..

#soup = BeautifulSoup(page, 'html.parser')
#print(soup)



#https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python
from requests_html import HTMLSession, AsyncHTMLSession

# session = HTMLSession()
# r = session.get(bolig_url)
# r.html.render()


import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()


session = AsyncHTMLSession()
r = await session.get(bolig_url)
#await r.html.arender()
#resp=r.html.raw_html
#print(resp)